In [1]:
#####
# 1.最新四半期決算の営業利益率は低い
# 2.最新四半期決算の売上高成長率が10%以上
# 3.最新四半期決算の対売上高営業利益成長率が20%以上
# 4.次期決算を、以下の方法で予測した場合において、営業利益率が前年同期比2倍以上になる
# --> 最新四半期決算の売上高成長率と対売上高営業利益成長率を、次期前年四半期決算に適用して次期決算を予想する

In [4]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [5]:
KPL = KessanPl()
KPL.with_columns_lastyear_settlement()
df1 = KPL.df.filter(pl.col("settlement_type")=="四")
df1 = df1.with_columns([
    pl.col("settlement_date").alias("key")
])

In [6]:
# para
valuation_date = date(2024, 8, 1)

KPL = KessanPl()
df2 = KPL.get_settlement_forcast_by_diff_growth_rate(valuation_date)
df2 = df2.filter(pl.col("settlement_type")=="四")
df2 = df2.with_columns([
    pl.col("nxt_settlement_date").alias("key")
])

In [7]:
KPL = KessanPl()
df3 = KPL.get_current_settlements(valuation_date)
df3 = df3.with_columns([
    pl.col("settlement_date").alias("key")
])

In [8]:
KPL = KessanPl()
df = KPL.get_settlement_updown_rate("四")
df = df.with_columns([
    pl.col("settlement_date").alias("key")
])
df4 = df.select(["code", "key", "updown_rate", "nh_updown_rate"])


In [9]:
# df1, df2, df3, df4をjoin&select
df = df2.join(df1, on=["code", "key"], how="left")
df21 = df

df = df21.with_columns([
    pl.col("settlement_date").alias("lst_settlement_date"),
    pl.col("announcement_date").alias("lst_announcement_date"),
    pl.col("settlement_type").alias("lst_settlement_type"),
    pl.col("sales").alias("lst_sales"),
    pl.col("operating_income").alias("lst_operating_income"),
    pl.col("ordinary_profit").alias("lst_ordinary_profit"),
    pl.col("final_profit").alias("lst_final_profit"),
    pl.col("quater").alias("lst_quater")
])
df = df.select([
    "code",	
    "lst_settlement_date",
    "lst_announcement_date",
    "lst_settlement_type",
    'lst_sales',
    'lst_operating_income',
    'lst_ordinary_profit',
    'lst_final_profit',
    "quater",
    "ly_settlement_date",
    "ly_sales",
    "ly_operating_income",
    "ly_ordinary_profit",
    "ly_final_profit",
    "nxt_settlement_date",
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",	
    "fcst_dgr_final_profit",
    "key"
])
df = df.join(df3, on=["code", "key"], how="left")
df = df.drop_nulls()

df123 = df

In [10]:
# ly_fcstの売上高、営業利益成長率を
# 列名ly_fcst_sales_gr, ly_fcst_operating_income_grで追加
df = df123.with_columns([
    (pl.lit(100) * (pl.col("fcst_dgr_sales") - pl.col("ly_sales")) / pl.col("ly_sales")).round(2).alias("ly_fcst_sales_gr"),
    (pl.lit(100) * (pl.col("fcst_dgr_operating_income") - pl.col("ly_operating_income")) / pl.col("ly_operating_income")).round(2).alias("ly_fcst_operating_income_gr"),
])

df = df.join(df4, on=["code", "key"], how="left")
df = df.drop_nulls()

df = df.sort(by=["ly_fcst_operating_income_gr"], descending=[True])
original_df = df
original_df

code,lst_settlement_date,lst_announcement_date,lst_settlement_type,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_settlement_date,ly_sales,ly_operating_income,ly_ordinary_profit,ly_final_profit,nxt_settlement_date,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,key,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater_right,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,date,str,i64,i64,i64,i64,i64,date,i64,i64,i64,i64,date,i64,i64,i64,i64,date,date,str,date,i64,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64
4406,2024-03-31,2024-05-10,"""四""",8783,456,476,303,4,2023-06-30,7952,1,183,126,2024-06-30,9986,1227,1205,896,2024-06-30,2024-06-30,"""四""",2024-08-09,8194,203,281,236,6.4,2.5,1,25.58,122600.0,4.8,-7.12
9438,2024-03-31,2024-05-09,"""四""",6892,617,726,928,2,2023-06-30,6500,1,-36,821,2024-06-30,7271,1227,1206,1633,2024-06-30,2024-06-30,"""四""",2024-08-06,7019,700,845,685,12.5,10.0,3,11.86,122600.0,3.5,-11.05
4418,2024-03-31,2024-05-10,"""四""",5531,41,28,8,3,2023-06-30,452,1,-29,-19,2024-06-30,55022,833,721,651,2024-06-30,2024-06-30,"""四""",2024-08-13,5292,4,-12,-213,-16.1,0.1,4,12073.01,83200.0,-13.7,-4.39
4418,2024-03-31,2024-05-10,"""四""",5531,41,28,8,3,2023-06-30,452,1,-29,-19,2024-06-30,55022,833,721,651,2024-06-30,2024-06-30,"""本""",2024-08-13,16457,50,-12,-278,-21.0,0.0,4,12073.01,83200.0,-13.7,-4.39
6145,2024-03-31,2024-05-10,"""四""",11465,2173,2165,1395,4,2023-06-30,4195,9,72,17,2024-06-30,16383,3727,3852,2320,2024-06-30,2024-06-30,"""四""",2024-08-09,4319,-611,-484,-345,-19.2,-14.1,1,290.54,41311.11,-27.0,-7.12
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5967,2024-05-31,2024-07-11,"""四""",2568,684,702,539,4,2023-08-31,1305,-5,52,18,2024-08-31,2917,922,956,779,2024-08-31,2024-08-31,"""四""",2024-10-11,1475,35,10,-20,-1.9,2.4,1,123.52,-18540.0,17.5,-3.89
1961,2024-03-31,2024-05-10,"""四""",72309,5588,5770,4090,4,2023-06-30,40727,-18,359,339,2024-06-30,80263,7083,7472,4527,2024-06-30,2024-06-30,"""四""",2024-08-09,48527,1540,2087,1345,25.3,3.2,1,97.08,-39450.0,2.6,-7.12
7494,2024-03-31,2024-05-15,"""四""",19278,1143,1145,653,2,2023-06-30,16144,-2,-165,-201,2024-06-30,19104,1514,1457,811,2024-06-30,2024-06-30,"""四""",2024-08-14,14560,-599,-696,-829,-28.4,-4.1,3,18.33,-75800.0,-28.4,-5.86


In [8]:
######
###### filter
######

In [ ]:
df = original_df
df.sort